<a href="https://colab.research.google.com/github/Lutambwe186/NLP/blob/main/Jonathan_NLP_Workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq unpackai
# !pip install -q git+https://github.com/unpackai/unpackai
!pip install -Uqq transformers==4.10.2
!pip install -q datasets transformers[sentencepiece]

     |████████████████████████████████| 73 kB 3.1 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 197 kB 66.4 MB/s 
     |████████████████████████████████| 59 kB 9.5 MB/s 
     |████████████████████████████████| 976 kB 58.4 MB/s 
     |████████████████████████████████| 2.8 MB 44.5 MB/s 
     |████████████████████████████████| 3.3 MB 59.6 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 880 kB 63.3 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
     |████████████████████████████████| 346 kB 28.7 MB/s 
     |████████████████████████████████| 86 kB 7.9 MB/s 
     |████████████████████████████████| 212 kB 76.6 MB/s 
     |████████████████████████████████| 140 kB 79.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 127 kB 65.8 MB/s 
     |████████████████████████████████| 144 kB 76.4 MB/s 
     |██████████████████

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModel,
    pipeline,
    set_seed,
    Trainer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    AutoModelForSequenceClassification,
)
from unpackai.nlp import Textual, InterpEmbeddingsTokenizer
from ipywidgets import interact
import logging
from fastai import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step One - Define a ML problem and propose a solution.

#### 1. Define the objective
**Your objective:** Train a model that is able to write a news story 
#### 2. Describe your dataset
**Your dataset:** The dataset consists of 2225 documents from the BBC news website corresponding to stories in five topical areas from 2004-2005
#### 3. Describe your model
**Your model:** a model that generates a 100 word news article.

### 4. Credit 
D. Greene and P. Cunningham. "Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering", Proc. ICML 2006.

In [ ]:
textual = Textual.from_path("/content/drive/MyDrive/BBC txt/final")
textual

Text (7983665 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual()

interactive(children=(IntSlider(value=100, description='page', max=200), Output()), _dom_classes=('widget-inte…

### Step Three - Model Training

We skip the Data Transformation as the Tokenization and Numericalisation happens during the Model Training.

In [ ]:
pretrained_model = pipeline("text-generation", model='gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
arguments = TrainingArguments(
    output_dir="./write_style",
    overwrite_output_dir=True,  
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=64,
    )

In [ ]:
trainer = textual.get_hf_trainer(
    model=pretrained_model.model,
    tokenizer = pretrained_model.tokenizer,
    arguments = arguments
    )

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (1369070 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 8030
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 1005


Step,Training Loss
500,3.435600
1000,3.206300


Saving model checkpoint to ./write_style/checkpoint-500
Configuration saved in ./write_style/checkpoint-500/config.json
Model weights saved in ./write_style/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./write_style/checkpoint-1000
Configuration saved in ./write_style/checkpoint-1000/config.json
Model weights saved in ./write_style/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1005, training_loss=3.320188517594219, metrics={'train_runtime': 1924.8847, 'train_samples_per_second': 12.515, 'train_steps_per_second': 0.522, 'total_flos': 1573631262720000.0, 'train_loss': 3.320188517594219, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to ./write_style
Configuration saved in ./write_style/config.json
Model weights saved in ./write_style/pytorch_model.bin


### Step Four - Interpret the model and generate text.

In [ ]:
style_writer = pipeline('text-generation',
                        model='./write_style',
                        tokenizer=pretrained_model.tokenizer
                        )

loading configuration file ./write_style/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 50,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.10.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading configurati

In [ ]:
style_writer("The queen visits",
            max_length=100,
            num_return_sequences=1)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The queen visits in public at her request to share her family and take her to the family\'s favorite movie theatre, The King\'s Speech, which is currently in its 70th year. She spends most of her time playing cards and holding hands. Sometimes she spends time reading poetry.\n\nWhile she enjoys talking to people and getting to know them, she does not like to be the subject of ridicule.\n\n"It does not bother me about myself," she said.'}]